In [9]:
import os
import pandas as pd

# centro_replace = {
#     'GUIA': '8811455',
#     'AVEIRO': '8811417',
#     'BRAGA': '8817018',
#     'COIMBRA': '8818171',
#     'LEIRIA': '8811456',
#     'MEM_MARTINS': '8811373',
#     'PERAFITA': '8818154',
#     'VISEU': '8811457',
#     'MARL': '8811496',
#     'PALMELA': '8819009',
#     'DEVESAS': '8819001',
#     'PINHEIRO_DE_FORA': '8819002',
# }

centro_replace = {
    'AVEIRO': '8811417',
}

for centro, co_value in centro_replace.items():
    Nome = centro  # Assuming you want to assign the centro name to Nome
    co = co_value


    if os.name == 'nt':  # Windows
        one_drive_folder = r'C:\Users\1420844\OneDrive - CTT - Correios de Portugal\01. Last-mile\06. Otimização Last Mile Expresso\Python\04.Estudos_km\02.Resultados\8818171'
    else:  # Assuming Unix-like (Linux, macOS)
        one_drive_folder = "/Users/rubenfilipemartinsdarocha/Downloads/Resultados/" + co




    contents = os.listdir(one_drive_folder)

    dataframes = []


    for item in contents:
        if item.endswith(".xlsx"):
            
                file_path = os.path.join(one_drive_folder, item)
                
                dataframe = pd.read_excel(file_path)
                file_name = os.path.basename(file_path)

                dataframe['File_Name'] = file_name

                dataframe["New_Column"] = ""
                previous_value = None

                for index, row in dataframe.iterrows():
                    if len(str(row["Chegada"])) == 13:
                        dataframe.at[index, "New_Column"] = row["Nome"]
                        previous_value = row["Nome"]
                    else:
                        dataframe.at[index, "New_Column"] = previous_value
                        

                def convert_datetime(value):
                    if pd.isnull(value):
                        return pd.NaT
                    try:
                        if "+1" in value:
                            return pd.NaT
                        return pd.to_datetime(value, format="%H:%M")
                    except ValueError:
                        return pd.NaT

                dataframe["Janela Horária"] = dataframe["Janela Horária"].str[:5]
                dataframe["Janela Horária"] = dataframe["Janela Horária"].apply(convert_datetime)

                dataframe["Min_Tempo_de_Inicio"] = dataframe.groupby("New_Column")["Janela Horária"].transform("min")
                dataframe["Max_Tempo_de_Inicio"] = dataframe.groupby("New_Column")["Janela Horária"].transform("max")
                filtered_dataframe = dataframe[dataframe["Chegada"].apply(lambda x: len(str(x)) == 13)]

                filtered_dataframe.rename(columns={
                    "Min_Tempo_de_Inicio": "Hora_primeiro_serviço",
                    "Max_Tempo_de_Inicio": "Hora_ultimo_serviço"
                }, inplace=True)

                desired_columns = [
                    "Nome", "QtdEnt", "Distância", "File_Name",
                    "Hora_primeiro_serviço", "Hora_ultimo_serviço"
                ]
                filtered_dataframe = filtered_dataframe[desired_columns]
                filtered_dataframe["Centro"] = filtered_dataframe["File_Name"].str.extract(r"(\d+)_(\d+)")[0]
                filtered_dataframe["Date"] = filtered_dataframe["File_Name"].str.extract(r"_(\d+)_(\d+)_")[1]

                filtered_dataframe.drop(columns=["File_Name"], inplace=True)

                filtered_dataframe["Hora_primeiro_serviço"] = filtered_dataframe["Hora_primeiro_serviço"].dt.strftime("%H:%M")
                filtered_dataframe["Hora_ultimo_serviço"] = filtered_dataframe["Hora_ultimo_serviço"].dt.strftime("%H:%M")
                
                ##--##
                
                        
                dataframe['horário'] = dataframe.apply(lambda row: str(row['Chegada'])[:5] if pd.isnull(row['Tempo de Início de Serviço']) else row['Tempo de Início de Serviço'], axis=1)
                dataframe = dataframe[~dataframe['horário'].str.contains(r'[^\d:]')]

                # Calculate the difference between the minimum value after and maximum value before each row
                dataframe['Min'] = dataframe['horário']
                dataframe['Min_After'] = dataframe['horário'].shift(-1)
                dataframe['Max_Before'] = dataframe['horário'].shift(1)
                dataframe['Max'] = dataframe['horário']
                
                dataframe['Min_After'] = pd.to_datetime(dataframe['Min_After'], format='%H:%M', errors='coerce')
                dataframe['Max_Before'] = pd.to_datetime(dataframe['Max_Before'], format='%H:%M', errors='coerce')

                dataframe['Difference_Min_After'] = (dataframe['Min_After'] - pd.to_datetime(dataframe['Min'], format='%H:%M')).dt.total_seconds() / 60
                dataframe['Difference_Max_Before'] = (pd.to_datetime(dataframe['Max'], format='%H:%M') - dataframe['Max_Before']).dt.total_seconds()  / 60

                dataframe['Difference_Min_After'].fillna(0, inplace=True)
                dataframe['Difference_Max_Before'].fillna(0, inplace=True)

                condition1 = (dataframe['Nome'].str.len() == 5) & (dataframe['Chegada'].str.len() == 13)
                condition2 = (dataframe['Nome'].str.len() == 5) & (dataframe['Chegada'].str.len() == 5)

                filtered_df1 = dataframe[condition1]
                filtered_df2 = dataframe[condition2]

                # Calculate Difference_Min_After and Difference_Max_Before for the filtered DataFrames
                filtered_df1['START'] = filtered_df1.groupby('New_Column')['Difference_Min_After'].transform('min')
                filtered_df1['START'] = filtered_df1['START'].apply(lambda x: '{:02}:{:02}'.format(int(x // 60), int(x % 60)))
                filtered_df2['END'] = filtered_df2.groupby('New_Column')['Difference_Max_Before'].transform('max')
                filtered_df2['END'] = filtered_df2['END'].apply(lambda x: '{:02}:{:02}'.format(int(x // 60), int(x % 60)))
                
                filtered_df1 = filtered_df1[['Nome', 'START']]
                filtered_df2 = filtered_df2[['Nome', 'END']]
                result = filtered_df1.merge(filtered_df2, on='Nome', suffixes=('_START', '_END'))

                
                FINAL = filtered_dataframe.merge(result, on='Nome')
                            
                
                # Convert 'START' and 'END' columns to Timedelta objects
                FINAL['START'] = pd.to_datetime(FINAL['START'], format='%H:%M')
                #FINAL['END'] = pd.to_datetime(FINAL['END'], format='%H:%M')
                
                FINAL['Hora_primeiro_serviço'] = pd.to_datetime(FINAL['Hora_primeiro_serviço'], format='%H:%M')
                #FINAL['Hora_ultimo_serviço'] = pd.to_datetime(FINAL['Hora_ultimo_serviço'], format='%H:%M')
                
                FINAL['END'] = FINAL['END'].str.split(':').apply(lambda x: pd.Timedelta(hours=int(x[0]), minutes=int(x[1])))
                FINAL['Hora_ultimo_serviço'] = FINAL['Hora_ultimo_serviço'].str.split(':').apply(lambda x: pd.Timedelta(hours=int(x[0]), minutes=int(x[1])))

                
                FINAL['END'] = pd.to_timedelta(FINAL['END'])
                FINAL['Hora_ultimo_serviço'] = pd.to_timedelta(FINAL['Hora_ultimo_serviço'])
                
                # Calculate 'Hora_inicio' and 'Hora_fim'
                FINAL['Hora_inicio'] = FINAL['Hora_primeiro_serviço'] - FINAL['START']
                FINAL['Hora_fim'] = FINAL['Hora_ultimo_serviço'] + FINAL['END']
                
                # Convert Timedelta columns to strings in the desired format
                FINAL['Hora_inicio'] = FINAL['Hora_inicio'].astype(str)
                FINAL['Hora_fim'] = FINAL['Hora_fim'].astype(str)

                # Reorder columns
                column_order = [
                    'Nome', 'QtdEnt', 'Distância', 'Hora_inicio',
                    'Hora_primeiro_serviço', 'Hora_ultimo_serviço',
                    'Hora_fim', 'Centro', 'Date'
                ]

                FINAL = FINAL[column_order]
                
                #break
                FINAL['Hora_inicio_str'] = FINAL['Hora_inicio'].astype(str)
                FINAL['Hora_primeiro_serviço_str'] = FINAL['Hora_primeiro_serviço'].astype(str)
                FINAL['Hora_ultimo_serviço_str'] = FINAL['Hora_ultimo_serviço'].astype(str)
                FINAL['Hora_fim_str'] = FINAL['Hora_fim'].astype(str)
                
                FINAL['Hora_inicio'] = FINAL['Hora_inicio_str'].str.extract(r'(\d{2}:\d{2}:\d{2})')
                FINAL['Hora_primeiro_serviço'] = FINAL['Hora_primeiro_serviço_str'].str.extract(r'(\d{2}:\d{2}:\d{2})')
                FINAL['Hora_ultimo_serviço'] = FINAL['Hora_ultimo_serviço_str'].str.extract(r'(\d{2}:\d{2}:\d{2})')
                FINAL['Hora_fim'] = FINAL['Hora_fim_str'].str.extract(r'(\d{2}:\d{2}:\d{2})')
                
                FINAL.drop('Hora_inicio_str', axis=1, inplace=True)
                FINAL.drop('Hora_primeiro_serviço_str', axis=1, inplace=True)
                FINAL.drop('Hora_ultimo_serviço_str', axis=1, inplace=True)
                FINAL.drop('Hora_fim_str', axis=1, inplace=True)
                
                
                dataframes.append(FINAL)
        
    combined_dataframe = pd.concat(dataframes, ignore_index=True)

    #combined_dataframe.to_excel('Final.xlsx', index=False)


    output_excel_path = os.path.join("/Users/rubenfilipemartinsdarocha/Downloads/", f"{co}_{Nome}.xlsx")
    combined_dataframe.to_excel(output_excel_path, index=False)

    print(f"DataFrame saved to {output_excel_path}")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/openpyxl/styles/stylesheet.py:226: UserWarning: Workbook contains no default style, apply openpyxl's default
  warn("Workbook contains no default style, apply openpyxl's default")
/var/folders/9n/zs_bd05s47xdjcj6_67tkw7c0000gn/T/ipykernel_5473/9878896.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dataframe.rename(columns={
/var/folders/9n/zs_bd05s47xdjcj6_67tkw7c0000gn/T/ipykernel_5473/9878896.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df1['START'] 

DataFrame saved to /Users/rubenfilipemartinsdarocha/Downloads/8811417_AVEIRO.xlsx


/var/folders/9n/zs_bd05s47xdjcj6_67tkw7c0000gn/T/ipykernel_5473/9878896.py:79: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_dataframe.rename(columns={
/var/folders/9n/zs_bd05s47xdjcj6_67tkw7c0000gn/T/ipykernel_5473/9878896.py:125: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df1['START'] = filtered_df1.groupby('New_Column')['Difference_Min_After'].transform('min')
/var/folders/9n/zs_bd05s47xdjcj6_67tkw7c0000gn/T/ipykernel_5473/9878896.py:126: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using 